# Standardizing Event Data for DeRisk

DeRisk currently works with event data in a raw format where relevant data (like the user, token, amount, etc.) is saved in a dictionary or list-like structure in one of the columns. To better extract information from the database, it is helpful to have a unified data structure where all relevant pieces of information are saved in separate columns. This allows for easy querying of all events of a given type (deposit, withdrawal, liquidation) for a given user, or for a lending protocol.

The following steps outline the process to take a sample of events and convert them to a standardized format that can be used to store information about any type of event and for any lending protocol.


In [1]:
#import libraries
import pandas as pd 

#display settings
pd.set_option('display.max_colwidth', None)

## 1.  Load event data  the paraquet file

In [14]:
def load_protocol_data(protocols: list[str]) -> pd.DataFrame:
    """
    Load data from Google Storage for the specified protocols and combine them into a single DataFrame.
    
    Parameters:
    protocols (list[str]): A list of protocol names to load data for.
    
    Returns:
    pd.DataFrame: A DataFrame containing the combined data from all specified protocols.
    
    """    
    combined_protocols_df = pd.DataFrame()
    
    for protocol in protocols:
        # Read from google storage
        url = f"https://storage.googleapis.com/derisk-persistent-state/{protocol}_data/events_sample.parquet"      
        print(f"Processing {protocol} from Google Storage...")
        df_protocol = pd.read_parquet(url)
        
        # Read from SQL DB
        ## Uncomment the following code block and comment the above three lines to load data from a local database
        # connection = src.db.establish_connection()
        # query = f"SELECT * FROM {protocol}_data" # Ensure table name is correct
        # print(f"Processing {protocol} from local database...")
        # df_protocol = pd.read_sql(query, con = connection)
        # connection.close()
        
        df_protocol['Protocol'] = protocol
        combined_protocols_df = pd.concat([combined_protocols_df, df_protocol], ignore_index=True)
    return combined_protocols_df

# List of protocols
PROTOCOLS = ['zklend', 'nostra_alpha', 'nostra_mainnet', 'hashstack_v0', 'hashstack_v1']

# Load the data
events = load_protocol_data(PROTOCOLS)
print(f"Combined dataframe shape: {events.shape}")

Processing zklend from Google Storage...
Processing nostra_alpha from Google Storage...
Processing nostra_mainnet from Google Storage...
Processing hashstack_v0 from Google Storage...
Processing hashstack_v1 from Google Storage...
Combined dataframe shape: (305033, 11)


In [15]:
events.head()

,block_hash,block_number,transaction_hash,event_index,from_address,keys,data,timestamp,key_name,Protocol,order
0,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,12,0x04c0a5193d58f74fbace4b74dcf65481e734ed1714121bdc571da345540efa05,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e8456f474e0abb1ae8d017da""]'","b'[""0x3fe2b97c1fd336e750087d68b9b867997fd64a2661ff3ca5a7c771641e8e7ac"",""0x33b2e3c9fd0803ce8000000"",""0x33b2e3c9fd0803ce8000000""]'",1683226849,AccumulatorsSync,zklend,NaN
1,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,15,0x04c0a5193d58f74fbace4b74dcf65481e734ed1714121bdc571da345540efa05,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e8456f474e0abb1ae8d017da""]'","b'[""0x68f5c6a61780768455de69077e07e89787839bf8166decfbf92b645209c0fb8"",""0x33b2e3c9fd0803ce8000000"",""0x33b2e3c9fd0803ce8000000""]'",1683226849,AccumulatorsSync,zklend,NaN
2,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,18,0x04c0a5193d58f74fbace4b74dcf65481e734ed1714121bdc571da345540efa05,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e8456f474e0abb1ae8d017da""]'","b'[""0xda114221cb83fa859dbdb4c44beeaa0bb37c7537ad5ae66fe5e0efd20e6eb3"",""0x33b2e3c9fd0803ce8000000"",""0x33b2e3c9fd0803ce8000000""]'",1683226849,AccumulatorsSync,zklend,NaN
3,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,6,0x04c0a5193d58f74fbace4b74dcf65481e734ed1714121bdc571da345540efa05,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e8456f474e0abb1ae8d017da""]'","b'[""0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7"",""0x33b2e3c9fd0803ce8000000"",""0x33b2e3c9fd0803ce8000000""]'",1683226849,AccumulatorsSync,zklend,NaN
4,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,9,0x04c0a5193d58f74fbace4b74dcf65481e734ed1714121bdc571da345540efa05,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e8456f474e0abb1ae8d017da""]'","b'[""0x53c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8"",""0x33b2e3c9fd0803ce8000000"",""0x33b2e3c9fd0803ce8000000""]'",1683226849,AccumulatorsSync,zklend,NaN


## 2. Define a function to transform Each Row
Define a function that decodes the keys and data columns, converts the hexadecimal strings to integers, and structures the information into a standardized format.

### For zklend Protocol, these are the events and their relevant data.
- for accumulators_sync event: token is in data[0], lending_accumulator is in data[1], debt_accumulator is in data[2]
- for deposit event: user is in data[0], token is in data[1], face_amount is in data[2]
- for collateral_enabled event: user is in data[0], token is in data[1]
- for collateral_disabled event: user is in data[0], token is in data[1]
- for withdrawal event: user is in data[0], token is in data[1], face_amount is in data[2]
- for borrowing event: user is in data[0], token is in data[1], raw_amount is in data[2], face_amount is in data[3]
- for repayment event: user is in data[1], token is in data[2], raw_amount is in data[3]
- for liquidation event: user is in data[1], debt_token is in data[2], debt_raw_amnt is in data[3], collateral_token is in data[5], collateral_face_amnt is in data[6]

In [17]:
# Define a function to decode bytes, convert to list, and extract specific elements based on the event
def hex_to_int(hex_str: str) -> int:
        return int(hex_str, 16)

def extract_zklend(row: pd.Series) -> pd.Series:
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
        
        if row['key_name'] == 'Deposit':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1],
                'face_amount': hex_to_int(data_list[2])
            })
        elif row['key_name'] == 'Withdrawal':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1],
                'face_amount': hex_to_int(data_list[2])
            })
        elif row['key_name'] == 'Liquidation':
            return pd.Series({
                'user': data_list[1],
                'debt_token': data_list[2],
                'debt_raw_amount': hex_to_int(data_list[3]),
                'collateral_token': data_list[5],
                'collateral_face_amount': data_list[6]
            })
        elif row['key_name'] == 'CollateralEnabled':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1]
            })
        elif row['key_name'] == 'CollateralDisabled':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1]
            })
        elif row['key_name'] == 'Borrowing':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1],
                'raw_amount': hex_to_int(data_list[2]),
                'face_amount': hex_to_int(data_list[3])
            })
        elif row['key_name'] == 'Repayment':
            return pd.Series({
                'user': data_list[1],
                'token': data_list[2],
                'raw_amount': hex_to_int(data_list[3])
            })
        elif row['key_name'] == 'AccumulatorsSync':
            return pd.Series({
                'token': data_list[0],
                'lending_accumulator': data_list[1],
                'debt_accumulator': data_list[2]
            })
        else:
            raise ValueError(f"Unrecognized key_name: {row['key_name']}") # Raising an error for unrecognized key_name
    except Exception as e:
        # Handle any parsing errors
        raise ValueError(f"Failed to extract data from row: {str(e)}") from e

# Filter the dataset for 'zklend' protocol
zklend_events = events[events['Protocol'] == 'zklend']

# Apply the extraction function to each row
standardized_zklend_events = zklend_events.apply(extract_zklend, axis=1)

# Concatenate the extracted elements with the original dataframe
standardized_zklend_events = pd.concat([zklend_events[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], standardized_zklend_events], axis=1)

# display the dataframe
standardized_zklend_events.head()

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'standardized_zklend_events.csv'
# standardized_zklend_events.to_csv(output_file_path, index=False)

,block_hash,block_number,transaction_hash,timestamp,key_name,collateral_face_amount,collateral_token,debt_accumulator,debt_raw_amount,debt_token,face_amount,lending_accumulator,raw_amount,token,user
0,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,1683226849,AccumulatorsSync,NaN,NaN,0x33b2e3c9fd0803ce8000000,NaN,NaN,NaN,0x33b2e3c9fd0803ce8000000,NaN,0x3fe2b97c1fd336e750087d68b9b867997fd64a2661ff3ca5a7c771641e8e7ac,NaN
1,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,1683226849,AccumulatorsSync,NaN,NaN,0x33b2e3c9fd0803ce8000000,NaN,NaN,NaN,0x33b2e3c9fd0803ce8000000,NaN,0x68f5c6a61780768455de69077e07e89787839bf8166decfbf92b645209c0fb8,NaN
2,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,1683226849,AccumulatorsSync,NaN,NaN,0x33b2e3c9fd0803ce8000000,NaN,NaN,NaN,0x33b2e3c9fd0803ce8000000,NaN,0xda114221cb83fa859dbdb4c44beeaa0bb37c7537ad5ae66fe5e0efd20e6eb3,NaN
3,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,1683226849,AccumulatorsSync,NaN,NaN,0x33b2e3c9fd0803ce8000000,NaN,NaN,NaN,0x33b2e3c9fd0803ce8000000,NaN,0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7,NaN
4,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,1683226849,AccumulatorsSync,NaN,NaN,0x33b2e3c9fd0803ce8000000,NaN,NaN,NaN,0x33b2e3c9fd0803ce8000000,NaN,0x53c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8,NaN


### For nostra_alpha protocol, the events are mapped on tokens, here are the events.
- non_interest_bearing_collateral_mint_event: deposit
- non_interest_bearing_collateral_burn_event: withdrawal
- interest_bearing_collateral_mint_event: deposit
- interest_bearing_collateral_burn_event: withdrawal
- debt_mint_event: borrowing
- debt_burn_event : repayment


In [19]:
# Define the mapping for token addresses and their types
NOSTRA_ALPHA_TOKENS_TO_TOKEN_TYPES :dict[str, str] =  { 
    '0x0553cea5d1dc0e0157ffcd36a51a0ced717efdadd5ef1b4644352bb45bd35453': 'non_interest_bearing_collateral',
    '0x047e794d7c49c49fd2104a724cfa69a92c5a4b50a5753163802617394e973833': 'non_interest_bearing_collateral',
    '0x003cd2066f3c8b4677741b39db13acebba843bbbaa73d657412102ab4fd98601': 'non_interest_bearing_collateral',
    '0x04403e420521e7a4ca0dc5192af81ca0bb36de343564a9495e11c8d9ba6e9d17': 'non_interest_bearing_collateral',
    '0x06b59e2a746e141f90ec8b6e88e695265567ab3bdcf27059b4a15c89b0b7bd53': 'non_interest_bearing_collateral',
    '0x070f8a4fcd75190661ca09a7300b7c93fab93971b67ea712c664d7948a8a54c6': 'interest_bearing_collateral',
    '0x029959a546dda754dc823a7b8aa65862c5825faeaaf7938741d8ca6bfdc69e4e': 'interest_bearing_collateral',
    '0x055ba2baf189b98c59f6951a584a3a7d7d6ff2c4ef88639794e739557e1876f0': 'interest_bearing_collateral',
    '0x01ac55cabf2b79cf39b17ba0b43540a64205781c4b7850e881014aea6f89be58': 'interest_bearing_collateral',
    '0x00687b5d9e591844169bc6ad7d7256c4867a10cee6599625b9d78ea17a7caef9': 'interest_bearing_collateral',
    '0x040b091cb020d91f4a4b34396946b4d4e2a450dbd9410432ebdbfe10e55ee5e5': 'debt',
    '0x03b6058a9f6029b519bc72b2cc31bcb93ca704d0ab79fec2ae5d43f79ac07f7a': 'debt',
    '0x065c6c7119b738247583286021ea05acc6417aa86d391dcdda21843c1fc6e9c6': 'debt',
    '0x0362b4455f5f4cc108a5a1ab1fd2cc6c4f0c70597abb541a99cf2734435ec9cb': 'debt',
    '0x075b0d87aca8dee25df35cdc39a82b406168fa23a76fc3f03abbfdc6620bb6d7': 'debt'
}

NOSTRA_ALPHA_INTEREST_RATE_MODEL_ADDRESS: str = '0x03d39f7248fb2bfb960275746470f7fb470317350ad8656249ec66067559e892'
# Define the event type mapping based on the token type and key name
NOSTRA_ALPHA_TOKEN_TYPES_TO_EVENT_TYPES : dict[str, str] = {
    'non_interest_bearing_collateral': {
        'Mint': 'deposit',
        'Burn': 'withdrawal'
    },
    'interest_bearing_collateral': {
        'Mint': 'deposit',
        'Burn': 'withdrawal'
    },
    'debt': {
        'Mint': 'borrowing',
        'Burn': 'repayment'
    }
}

def get_nostra_alpha_event_type(token_address: str, key_name: str) ->str:
    # Special case for InterestStateUpdated and the specific interest rate model address
    if token_address == NOSTRA_ALPHA_INTEREST_RATE_MODEL_ADDRESS and key_name == 'InterestStateUpdated':
        return 'interest_rate_updated'

    token_type = NOSTRA_ALPHA_TOKENS_TO_TOKEN_TYPES.get(token_address)
    if token_type:
        event_type = NOSTRA_ALPHA_TOKEN_TYPES_TO_EVENT_TYPES.get(token_type, {}).get(key_name)
        if event_type:
            return event_type
    # If no match is found, raise an error
    raise ValueError(f"Unrecognized event type for token address {token_address} and key name {key_name}")

def extract_nostra_alpha(row: pd.Series) -> pd.Series:
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
        event_type = get_nostra_alpha_event_type(row['from_address'], row['key_name'])
        
        if row['key_name'] in ['Mint', 'Burn']:
            return pd.Series({
                'user': data_list[0],
                'token': row['from_address'],
                'face_amount': hex_to_int(data_list[1]),
                'event_type': event_type
            })
        elif row['key_name'] == 'InterestStateUpdated':
            return pd.Series({
                'debt_token': data_list[0],
                'lending_rate': data_list[1],
                'borrow_rate': data_list[3],
                'lend_index': data_list[5],
                'borrow_index': data_list[7],
                'event_type': 'interest_rate_updated'
                
            })
        else:
            raise ValueError(f"Unrecognized key_name: {row['key_name']}") # Raising an error for unrecognized key_name
    except Exception as e:
        # Handle any parsing errors
        raise ValueError(f"Failed to extract data from row: {str(e)}") from e

# Filter the dataset for 'nostra_alpha' protocol
nostra_alpha_events = events[events['Protocol'] == 'nostra_alpha']

# Apply the extraction function to each row
standardized_nostra_alpha_events = nostra_alpha_events.apply(extract_nostra_alpha, axis=1)

# Concatenate the extracted elements with the original dataframe
standardized_nostra_alpha_events = pd.concat([nostra_alpha_events[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], standardized_nostra_alpha_events], axis=1)

# Show a sample of the resulting dataframe
standardized_nostra_alpha_events.head()

# Uncomment the following to save the resulting dataframe to a new CSV file
# output_file_path = 'standardized_nostra_alpha_events.csv'
# standard_nalpha.to_csv(output_file_path, index=False)



,block_hash,block_number,transaction_hash,timestamp,key_name,borrow_index,borrow_rate,debt_token,event_type,face_amount,lend_index,lending_rate,token,user
100000,0x0365f72ec4f522f5008f47de643370d91f285be927499bb95dbda1ff7a84c5f2,10927,0x06ddd34767c8cef97d4508bcbb4e3771b1c93e160e02ca942cadbdfa29ef9ba8,1669193049,Mint,NaN,NaN,NaN,deposit,4.000000e+15,NaN,NaN,0x070f8a4fcd75190661ca09a7300b7c93fab93971b67ea712c664d7948a8a54c6,0x6810f5565965ac915feee7d57d3225cd470ce54a40b5b53d69d9f4ed59bd719
100001,0x0365f72ec4f522f5008f47de643370d91f285be927499bb95dbda1ff7a84c5f2,10927,0x06ddd34767c8cef97d4508bcbb4e3771b1c93e160e02ca942cadbdfa29ef9ba8,1669193049,InterestStateUpdated,0xde0b6b3a7640000,0x0,0x40b091cb020d91f4a4b34396946b4d4e2a450dbd9410432ebdbfe10e55ee5e5,interest_rate_updated,NaN,0xde0b6b3a7640000,0x0,NaN,NaN
100002,0x032328dc8371df35542516c3b99b3f845710dd5d3dc78bdb8bd76fe437582c50,10958,0x06c9b11388f448003ca5f8cff6f8e1dfafeef305042c057ba06c068bcb2cb93b,1669199979,Mint,NaN,NaN,NaN,deposit,5.000000e+06,NaN,NaN,0x029959a546dda754dc823a7b8aa65862c5825faeaaf7938741d8ca6bfdc69e4e,0x6810f5565965ac915feee7d57d3225cd470ce54a40b5b53d69d9f4ed59bd719
100003,0x032328dc8371df35542516c3b99b3f845710dd5d3dc78bdb8bd76fe437582c50,10958,0x06c9b11388f448003ca5f8cff6f8e1dfafeef305042c057ba06c068bcb2cb93b,1669199979,InterestStateUpdated,0xde0b6b3a7640000,0x0,0x3b6058a9f6029b519bc72b2cc31bcb93ca704d0ab79fec2ae5d43f79ac07f7a,interest_rate_updated,NaN,0xde0b6b3a7640000,0x0,NaN,NaN
100004,0x023d95223e40b917c74654cc2a3159e0cebbb4956ae51625571651d5a6c2e6a6,10961,0x0347a4f9206a0016f0f598e0042e785f1e5c1491d25a07004a7f450f41d5899c,1669200725,Mint,NaN,NaN,NaN,deposit,1.000000e+15,NaN,NaN,0x070f8a4fcd75190661ca09a7300b7c93fab93971b67ea712c664d7948a8a54c6,0x1b706b18846667c44fdf2dc302ed90b98c0ae3aa6e041fdbdea1289d0284b03


### For hashstack_v0 protocol these are the events and their relevant data:
- for new loan: loan_id is in data[0]int, user is in data[1], debt_token is in data[2], commitment is in data[3], debt_face_amount is in data[4]int, borrowed_collateral_token is in data[6],borrowed_collateral_face_amount is in data[7]int, is_loan_withdrawn is in data[9] debt_category is in data[10]int, state is in data[11], l3_integration is in data[12], created_at is in data[13], [`collateral`] `market` is in data[14], amount is in data[15], current_amount is in data[17], commitment is repeated in data_list[19]( but the values do not all match), timelock_validity is in data_list[20], is_timelock_activated is in data_list[21], activation_time is in data_list[22], timestamp is in data_list[23]
                
- for loan_withdrawal: id (loan_record): Located in data[0], owner: Located in data[1], market: Located in data[2], commitment: Located in data[3], amount: Located in data[4], current_market: Located in data[6], current_amount: Located in data[7], is_loan_withdrawn: Located in data[9], debt_category: Located in data[10], state: Located in data[11], l3_integration: Located in data[12], created_at: Located in data[13], amount_withdrawn: Located in data[14] (inside amount_withdrawn), timestamp: Located in data[16] (inside timestamp).

- for loan_interest_deducted: market (collateral_record): Located in data[0], amount: Located in data[1], (empty): Located in data[2] (not used), current_amount: Located in data[3], (empty): Located in data[4] (not used), commitment: Located in data[5], timelock_validity: Located in data[6], is_timelock_activated: Located in data[7], activation_time: Located in data[8], accrued_interest (inside accrued_interest): Located in data[9], (empty): Located in data[10] (not used), loan_id (inside loan_id): Located in data[11], amount_withdrawn (inside amount_withdrawn): Located in data[12], (empty): Located in data[13] (not used), timestamp (inside timestamp): Located in data[14]
  
- for loan_swap: old_loan_id : Located in data[0], old_loan_owner: Located in data[1], old_loan_market: Located in data[2], old_loan_commitment: Located in data[3], old_loan_amount: Located in data[4], (empty): Located in data[5] (not used), old_loan_current_market: Located in data[6], old_loan_current_amount: Located in data[7], (empty): Located in data[8] (not used), old_loan_is_loan_withdrawn: Located in data[9], old_loan_debt_category: Located in data[10], old_loan_state: Located in data[11], old_loan_l3_integration: Located in data[12], old_loan_created_at: Located in data[13], new_loan_id: Located in data[14], new_loan_owner: Located in data[15], new_loan_market: Located in data[16], new_loan_commitment: Located in data[17], new_loan_amount: Located in data[18], (empty): Located in data[19] (not used), new_loan_current_market: Located in data[20], new_loan_current_amount: Located in data[21], (empty): Located in data[22] (not used), new_loan_is_loan_withdrawn: Located in data[23], new_loan_debt_category: Located in data[24], new_loan_state: Located in data[25], new_loan_l3_integration: Located in data[26], new_loan_created_at: Located in data[27], timestamp: Located in data[28].

  
- for collateral_added: collateral_market: Located in data[0], collateral_amount: Located in data[1], (empty): Located in data[2] (not used), collateral_current_amount: Located in data[3], (empty): Located in data[4] (not used), collateral_commitment: Located in data[5], collateral_timelock_validity: Located in data[6], collateral_is_timelock_activated: Located in data[7], collateral_activation_time: Located in data[8], collateral_loan_id: Located in data[9], collateral_amount_added: Located in data[10], (empty): Located in data[11] (not used), collateral_timestamp: Located in data[12]
  
- for loan_repaid: loan_id: Located in data[0], loan_owner: Located in data[1], loan_market: Located in data[2], loan_commitment: Located in data[3], loan_amount: Located in data[4], (empty): Located in data[5] (not used), loan_current_market: Located in data[6], loan_current_amount: Located in data[7], (empty): Located in data[8] (not used), loan_is_loan_withdrawn: Located in data[9], loan_debt_category: Located in data[10], loan_state: Located in data[11], loan_l3_integration: Located in data[12], loan_created_at: Located in data[13], loan_timestamp: Located in data[14].

- for collateral_withdrawal: collateral_market: Located in data[0], collateral_amount: Located in data[1], (empty): Located in data[2] (not used), collateral_current_amount: Located in data[3], (empty): Located in data[4] (not used), collateral_commitment: Located in data[5], collateral_timelock_validity: Located in data[6], collateral_is_timelock_activated: Located in data[7], collateral_activation_time: Located in data[8], collateral_loan_id: Located in data[9], collateral_amount_withdrawn: Located in data[10], (empty): Located in data[11] (not used), collateral_timestamp: Located in data[12].

- for liquidated_event: loan_id is in data[0]int, user is in data[1], debt_token is in data[2], commitment is in data[3],amount is in data[4]int,  borrowed_collateral_token is in data[6], borrowed_collateral_face_amount is in data[7]int, loan_withdrawn is in data[9], debt_category is in data[10]int, state is in data[11], l3_integration is in data[12], created_at is in data[13], liquidator is in data[14], timestamp is in data[15]
  

In [21]:
def standardize_hashstack_v0_event(row: pd.Series) -> pd.Series:
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
        
        if row['key_name'] == 'new_loan':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'user': data_list[1],
                'debt_token': data_list[2],
                'commitment': data_list[3],
                'debt_face_amount': hex_to_int(data_list[4]),
                'borrowed_collateral_token': data_list[6],
                'borrowed_collateral_face_amount': hex_to_int(data_list[7]),
                'is_loan_withdrawn': data_list[9],
                'debt_category': hex_to_int(data_list[10]),
                'state': data_list[11],
                'l3_integration': data_list[12],
                'created_at': data_list[13],
                'collateral_market': data_list[14],
                'amount': data_list[15],
                'current_amount': data_list[17],
                'timelock_validity': data_list[20],
                'is_timelock_activated': data_list[21],
                'activation_time': data_list[22],
                'timestamp': row['timestamp']
            })
        elif row['key_name'] == 'loan_withdrawal':
            return pd.Series({
                'id': data_list[0],
                'owner': data_list[1],
                'market': data_list[2],
                'commitment': data_list[3],
                'amount': data_list[4],
                'current_market': data_list[6],
                'current_amount': data_list[7],
                'is_loan_withdrawn': data_list[9],
                'debt_category': data_list[10],
                'state': data_list[11],
                'l3_integration': data_list[12],
                'created_at': data_list[13],
                'amount_withdrawn': data_list[14],
                'timestamp': data_list[16]
            })
        elif row['key_name'] == 'loan_interest_deducted':
            return pd.Series({
                'market': data_list[0],
                'amount': data_list[1],
                'current_amount': data_list[3],
                'commitment': data_list[5],
                'timelock_validity': data_list[6],
                'is_timelock_activated': data_list[7],
                'activation_time': data_list[8],
                'accrued_interest': data_list[9],
                'loan_id': data_list[11],
                'amount_withdrawn': data_list[12],
                'timestamp': row['timestamp']
            })
        elif row['key_name'] == 'loan_swap':
            return pd.Series({
                'old_loan_id': data_list[0],
                'old_loan_owner': data_list[1],
                'old_loan_market': data_list[2],
                'old_loan_commitment': data_list[3],
                'old_loan_amount': data_list[4],
                'old_loan_current_market': data_list[6],
                'old_loan_current_amount': data_list[7],
                'old_loan_is_loan_withdrawn': data_list[9],
                'old_loan_debt_category': data_list[10],
                'old_loan_state': data_list[11],
                'old_loan_l3_integration': data_list[12],
                'old_loan_created_at': data_list[13],
                'new_loan_id': data_list[14],
                'new_loan_owner': data_list[15],
                'new_loan_market': data_list[16],
                'new_loan_commitment': data_list[17],
                'new_loan_amount': data_list[18],
                'new_loan_current_market': data_list[20],
                'new_loan_current_amount': data_list[21],
                'new_loan_is_loan_withdrawn': data_list[23],
                'new_loan_debt_category': data_list[24],
                'new_loan_state': data_list[25],
                'new_loan_l3_integration': data_list[26],
                'new_loan_created_at': data_list[27],
                'timestamp': data_list[28]
            })
        elif row['key_name'] == 'collateral_added':
            return pd.Series({
                'collateral_market': data_list[0],
                'collateral_amount': data_list[1],
                'collateral_current_amount': data_list[3],
                'collateral_commitment': data_list[5],
                'collateral_timelock_validity': data_list[6],
                'collateral_is_timelock_activated': data_list[7],
                'collateral_activation_time': data_list[8],
                'collateral_loan_id': data_list[9],
                'collateral_amount_added': data_list[10],
                'collateral_timestamp': data_list[12]
            })
        elif row['key_name'] == 'loan_repaid':
            return pd.Series({
                'loan_id': data_list[0],
                'loan_owner': data_list[1],
                'loan_market': data_list[2],
                'loan_commitment': data_list[3],
                'loan_amount': data_list[4],
                'loan_current_market': data_list[6],
                'loan_current_amount': data_list[7],
                'loan_is_loan_withdrawn': data_list[9],
                'loan_debt_category': data_list[10],
                'loan_state': data_list[11],
                'loan_l3_integration': data_list[12],
                'loan_created_at': data_list[13],
                'loan_timestamp': data_list[14]
            })
        elif row['key_name'] == 'collateral_withdrawal':
            return pd.Series({
                'collateral_market': data_list[0],
                'collateral_amount': data_list[1],
                'collateral_current_amount': data_list[3],
                'collateral_commitment': data_list[5],
                'collateral_timelock_validity': data_list[6],
                'collateral_is_timelock_activated': data_list[7],
                'collateral_activation_time': data_list[8],
                'collateral_loan_id': data_list[9],
                'collateral_amount_withdrawn': data_list[10],
                'collateral_timestamp': data_list[12]
            })
        elif row['key_name'] == 'liquidated':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'user': data_list[1],
                'debt_token': data_list[2],
                'commitment': data_list[3],
                'amount': hex_to_int(data_list[4]),
                'borrowed_collateral_token': data_list[6],
                'borrowed_collateral_face_amount': hex_to_int(data_list[7]),
                'loan_withdrawn': data_list[9],
                'debt_category': hex_to_int(data_list[10]),
                'state': data_list[11],
                'l3_integration': data_list[12],
                'created_at': data_list[13],
                'liquidator': data_list[14],
                'timestamp': data_list[15]
            })
        else:
            raise ValueError(f"Unrecognized key_name: {row['key_name']}") # Raising an error for unrecognized key_name
    except Exception as e:
        # Handle any parsing errors
        raise ValueError(f"Failed to extract data from row: {str(e)}") from e

# Filter the dataset for 'hashstack_v0' protocol
hashstack_v0_events = events[events['Protocol'] == 'hashstack_v0']

# Apply the extraction function to each row
standardized_hashstack_v0_events = hashstack_v0_events.apply(standardize_hashstack_v0_event, axis=1)

# Concatenate the extracted elements with the original dataframe
standardized_hashstack_v0_events = pd.concat([hashstack_v0_events[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], standardized_hashstack_v0_events], axis=1)

# Display the resulting dataframe
standardized_hashstack_v0_events.head()

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'standardized_hashstack_v0_events.csv'
# standardized_hashstack_v0_events.to_csv(output_file_path, index=False)

,block_hash,block_number,transaction_hash,timestamp,key_name,accrued_interest,activation_time,amount,amount_withdrawn,borrowed_collateral_face_amount,...,old_loan_is_loan_withdrawn,old_loan_l3_integration,old_loan_market,old_loan_owner,old_loan_state,owner,state,timelock_validity,timestamp,user
300000,0x07c8f77b0b9bb2b29fd3d3c729717b444813ab48a8e92b4b38d868b783e6cd8f,21506,0x05bb51553067fce9bed47b575dd2e7af6ed460a2ddcbd21f72680fb966b1bb53,1678096181,new_loan,NaN,0x6405b735,0x0,NaN,20979000000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,0x6e6f6e65,0x1,1678096181,0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f507b4ca728d8e427b7864
300001,0x0657249d2b3dff56c0715d8dbe01521098ac96ac4d20172e227900a3fdb049ed,21535,0x0103ce71c2dfbcec09b93f9f50a08255dc6fc17982f5e50d93360addf7d24ac3,1678122298,new_loan,NaN,0x64061d3a,0x0,NaN,17982000000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,0x6e6f6e65,0x0,1678122298,0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f507b4ca728d8e427b7864
300002,0x0657249d2b3dff56c0715d8dbe01521098ac96ac4d20172e227900a3fdb049ed,21535,0x07f08e47dab885ba7fd7c5e0313443e42ccf8d53c5ae52324d6ccc6167b25c43,1678122298,new_loan,NaN,0x64061d3a,0x0,NaN,29970000,...,NaN,NaN,NaN,NaN,NaN,NaN,0x6e6f6e65,0x1,1678122298,0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f507b4ca728d8e427b7864
300003,0x03951b3242291068d1456d3ff35c0f95c60c6d185d37f6c4d1400214122e56bb,21602,0x0448cfe386be1e6919ed40f7a602704043c9fb7ffad924c6785fcc991b5a109c,1678201433,new_loan,NaN,0x0,0x2386f26fc10000,NaN,29970000,...,NaN,NaN,NaN,NaN,NaN,NaN,0x1,0x0,1678201433,0x1e307316999c4830ada67b495d03246ab205e77853bb117da7b555122a33bc7
300004,0x0455454837ade622189fc590fcb911f9e6e08ea1669a09a3ec22ddb4773ab9df,21603,0x0606b7eb810306e25ad5eba69a56e3d6fe356021c1716cd1231957b0fd760066,1678203696,new_loan,NaN,0x0,0x2386f26fc10000,NaN,19980000000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,0x1,0x0,1678203696,0x74061d07a0fbd8ffc376dc4f593c69854c51b6c2fe5596d88452e9efdd76864


### for hashstack_v1 protocol, these are the following events and their relevant data:
- for new_loan event: loan_id is in data[0]int, borrower is in data[1], market is in data[2], loan_amount is in data[3]int, current_market is in data[5], current_amount is in data[6], state is in data[8], l3_integration is in data[9], l3_category is in data[10], loan_created_at is in data[11], collateral_loan_id is in data[12], collateral_token is in data[13], collateral_amount is in data[14]int, collateral_created_at is in data[16], timestamp is in data[17]
  
- for loan_spent: old_loan_id is in data[0]int, borrower is in data[1], market is in data[2], old_loan_amount is in data[3]int, current_market is in data[5], current_amount is in data[6], state is in data[8], l3_integration is in data[9], l3_category is in data[10], loan_created_at is in data[11], new_loan_id is in data[12], new_borrower is in data[13], new_market is in data[14], new_loan_amount is in data[15]int, new_current_market is in data[17], new_current_amount is in data[18], new_state is in data[20], new_l3_integration is in data[21], new_l3_category is in data[22], new_loan_created_at is in data[23], timestamp is in data[24]

- for collateral_added: loan_id is in data[0]int, collateral_token is in data[1], amount is in data[2], created_at is in data[4], amount_added is in data[5], timestamp is in data[6]

- for loan_repaid: loan_id is in data[0], borrower is in data[1], market is located in data[2], amount is located in data[3], current_market is located in data[5], current_amount is located in data[6], state is located in data[8], l3_integration is located in data[9], l3_category is located in data[10], created_at is located in data[11], new_ loan is located in data[12], new_borrower is located in data[13], new_market is in data[14], new_amount is in data[15], new_current_mark is in data[17], new_current_amount[18], new_state is in data[20], new_l3_integration is in data[21], new_l3_category is in data[22], new_created_at is in data[22], collateral_loan_id is in data[23], collateral_token is in data[24], collateral_amount is in data[25], collateral_created_at is in data[27], totalUserDebt is in data[28], timestamp is in data[29]
- for loan_transfered: loan_id is in data[0]int, sender is in data[1], reciever is in data[2], timestamp is in data[3]

In [12]:
def standardize_hashstack_v1_event(row: pd.Series) -> pd.Series:
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
        
        if row['key_name'] == 'new_loan':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'borrower': data_list[1],
                'market': data_list[2],
                'loan_amount': hex_to_int(data_list[3]),
                'current_market': data_list[5],
                'current_amount': data_list[6],
                'state': data_list[8],
                'l3_integration': data_list[9],
                'l3_category': data_list[10],
                'loan_created_at': data_list[11],
                'collateral_loan_id': hex_to_int(data_list[12]),
                'collateral_token': data_list[13],
                'collateral_amount': hex_to_int(data_list[14]),
                'collateral_created_at': data_list[16],
                'timestamp': data_list[17]
            })
        elif row['key_name'] == 'loan_spent':
            return pd.Series({
                'old_loan_id': hex_to_int(data_list[0]),
                'borrower': data_list[1],
                'market': data_list[2],
                'old_loan_amount': hex_to_int(data_list[3]),
                'current_market': data_list[5],
                'current_amount': data_list[6],
                'state': data_list[8],
                'l3_integration': data_list[9],
                'l3_category': data_list[10],
                'loan_created_at': data_list[11],
                'new_loan_id': data_list[12],
                'new_borrower': data_list[13],
                'new_market': data_list[14],
                'new_loan_amount': hex_to_int(data_list[15]),
                'new_current_market': data_list[17],
                'new_current_amount': data_list[18],
                'new_state': data_list[20],
                'new_l3_integration': data_list[21],
                'new_l3_category': data_list[22],
                'new_loan_created_at': data_list[23],
                'timestamp': data_list[24]
            })
        elif row['key_name'] == 'collateral_added':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'collateral_token': data_list[1],
                'amount': data_list[2],
                'created_at': data_list[4],
                'amount_added': data_list[5],
                'timestamp': data_list[6]
            })
        elif row['key_name'] == 'loan_repaid':
            return pd.Series({
                'loan_id': data_list[0],
                'borrower': data_list[1],
                'market': data_list[2],
                'amount': data_list[3],
                'current_market': data_list[5],
                'current_amount': data_list[6],
                'state': data_list[8],
                'l3_integration': data_list[9],
                'l3_category': data_list[10],
                'created_at': data_list[11],
                'new_loan_id': data_list[12],
                'new_borrower': data_list[13],
                'new_market': data_list[14],
                'new_amount': data_list[15],
                'new_current_market': data_list[17],
                'new_current_amount': data_list[18],
                'new_state': data_list[20],
                'new_l3_integration': data_list[21],
                'new_l3_category': data_list[22],
                'new_created_at': data_list[23],
                'collateral_loan_id': data_list[24],
                'collateral_token': data_list[25],
                'collateral_amount': data_list[26],
                'collateral_created_at': data_list[27],
                'totalUserDebt': data_list[28],
                'timestamp': data_list[29]
            })
        elif row['key_name'] == 'loan_transferred':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'sender': data_list[1],
                'receiver': data_list[2],
                'timestamp': data_list[3]
            })
        else:
            raise ValueError(f"Unrecognized key_name: {row['key_name']}") # Raising an error for unrecognized key_name
    except Exception as e:
        # Handle any parsing errors
        raise ValueError(f"Failed to extract data from row: {str(e)}") from e

# Filter the dataset for 'hashstack_v1' protocol
hashstack_v1_events = events[events['Protocol'] == 'hashstack_v1']

# Apply the extraction function to each row
standardized_hashstack_v1_events = hashstack_v1_events.apply(standardize_hashstack_v1_event, axis=1)

# Concatenate the extracted elements with the original dataframe
standardized_hashstack_v1_events = pd.concat([hashstack_v1_events[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], standardized_hashstack_v1_events], axis=1)

# Display the resulting dataframe
standardized_hashstack_v1_events.head()

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'standardized_hashstack_v1_events.csv'
# standardized_hashstack_v1_events.to_csv(output_file_path, index=False)

,block_hash,block_number,transaction_hash,timestamp,key_name,amount,amount_added,borrower,collateral_amount,collateral_created_at,...,new_loan_id,new_market,new_state,old_loan_amount,old_loan_id,receiver,sender,state,timestamp,totalUserDebt
300780,0x068b04151068a7acc1f72c9b2023e176037cb6bce171a5e3abaf6fb961f4e1a2,273262,0x0676e3ae850f6fae3da1269b9c390f74c26eb1948ca2bd1e16a47fa9399ca84d,1695977183,new_loan,NaN,NaN,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a,2000000,0x65168edf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x1,0x65168edf,NaN
300781,0x06c263286120af7afb020f535cd53e17da319df8435058d76450bdcd3ebaecb3,273300,0x06ebb5723abf3878214b66bd4bbe7087280707da7231b074a1bcfba8b5ec78d7,1695977763,new_loan,NaN,NaN,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a,2000000,0x65169123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x1,0x65169123,NaN
300782,0x07d0f1faf834b7906e4de54c440d0aa35112bc1a243c12e63e4f8b6006e857df,273372,0x0051f75ef1e08f70d1c8efe7866384d026aa0ca092ded8bd1c903aac0478b990,1695978978,loan_spent,NaN,NaN,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a,NaN,NaN,...,0x1,0x12b8185e237dd0340340faeb3351dbe53f8a42f5a9bf974ddf90ced56e301c7,0x2,4500000,1.0,NaN,NaN,0x1,0x651695e2,NaN
300783,0x06f650deb66bb602f02bd87f29aa7c28909b126f9d7c738f8a3ad200baa888c4,273387,0x01bf0af4c1922e2b8cc523e6e088d72885701e9416058a3e6bcc2bcf7f0bd5d0,1695979201,collateral_added,0x3d08fe,0x1e847e,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x0,NaN
300784,0x06384cf4b11477d048623b43e45970c5a4a10653706547b76e92096f7d421774,273396,0x0389888c9ef7dfe1491a3d15afa79b565bc52dccb01d512693f786f6c61fdc0f,1695979342,collateral_added,0x4c4b3e,0xf4240,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x0,NaN


### for nostra_mainnet, the same extraction will be applied as with nostra_alpha.

In [22]:
# Define mapping for their token and types
NOSTRA_MAINNET_TOKENS_TO_TOKEN_TYPES: dict[str, str] = {
    '0x01fecadfe7cda2487c66291f2970a629be8eecdcb006ba4e71d1428c2b7605c7': 'interest_bearing_deposit',
    '0x002fc2d4b41cc1f03d185e6681cbd40cced61915d4891517a042658d61cba3b1': 'interest_bearing_deposit',
    '0x0360f9786a6595137f84f2d6931aaec09ceec476a94a98dcad2bb092c6c06701': 'interest_bearing_deposit',
    '0x022ccca3a16c9ef0df7d56cbdccd8c4a6f98356dfd11abc61a112483b242db90': 'interest_bearing_deposit',
    '0x0735d0f09a4e8bf8a17005fa35061b5957dcaa56889fc75df9e94530ff6991ea': 'interest_bearing_deposit',
    '0x00ca44c79a77bcb186f8cdd1a0cd222cc258bebc3bec29a0a020ba20fdca40e9': 'interest_bearing_deposit',
    '0x0507eb06dd372cb5885d3aaf18b980c41cd3cd4691cfd3a820339a6c0cec2674': 'interest_bearing_deposit',
    '0x026c5994c2462770bbf940552c5824fb0e0920e2a8a5ce1180042da1b3e489db': 'interest_bearing_deposit',
    '0x078a40c85846e3303bf7982289ca7def68297d4b609d5f588208ac553cff3a18': 'interest_bearing_deposit',
    '0x044debfe17e4d9a5a1e226dabaf286e72c9cc36abbe71c5b847e669da4503893': 'non_interest_bearing_collateral',
    '0x05f296e1b9f4cf1ab452c218e72e02a8713cee98921dad2d3b5706235e128ee4': 'non_interest_bearing_collateral',
    '0x0514bd7ee8c97d4286bd481c54aa0793e43edbfb7e1ab9784c4b30469dcf9313': 'non_interest_bearing_collateral',
    '0x005c4676bcb21454659479b3cd0129884d914df9c9b922c1c649696d2e058d70': 'non_interest_bearing_collateral',
    '0x036b68238f3a90639d062669fdec08c4d0bdd09826b1b6d24ef49de6d8141eaa': 'non_interest_bearing_collateral',
    '0x05eb6de9c7461b3270d029f00046c8a10d27d4f4a4c931a4ea9769c72ef4edbb': 'non_interest_bearing_collateral',
    '0x02530a305dd3d92aad5cf97e373a3d07577f6c859337fb0444b9e851ee4a2dd4': 'non_interest_bearing_collateral',
    '0x040f5a6b7a6d3c472c12ca31ae6250b462c6d35bbdae17bd52f6c6ca065e30cf': 'non_interest_bearing_collateral',
    '0x0142af5b6c97f02cac9c91be1ea9895d855c5842825cb2180673796e54d73dc5': 'non_interest_bearing_collateral',
    '0x057146f6409deb4c9fa12866915dd952aa07c1eb2752e451d7f3b042086bdeb8': 'interest_bearing_collateral',
    '0x05dcd26c25d9d8fd9fc860038dcb6e4d835e524eb8a85213a8cda5b7fff845f6': 'interest_bearing_collateral',
    '0x0453c4c996f1047d9370f824d68145bd5e7ce12d00437140ad02181e1d11dc83': 'interest_bearing_collateral', 
    '0x04f18ffc850cdfa223a530d7246d3c6fc12a5969e0aa5d4a88f470f5fe6c46e9': 'interest_bearing_collateral',
    '0x05b7d301fa769274f20e89222169c0fad4d846c366440afc160aafadd6f88f0c': 'interest_bearing_collateral',
    '0x009377fdde350e01e0397820ea83ed3b4f05df30bfb8cf8055d62cafa1b2106a': 'interest_bearing_collateral',
    '0x0739760bce37f89b6c1e6b1198bb8dc7166b8cf21509032894f912c9d5de9cbd': 'interest_bearing_collateral',
    '0x07c2e1e733f28daa23e78be3a4f6c724c0ab06af65f6a95b5e0545215f1abc1b': 'interest_bearing_collateral',
    '0x067a34ff63ec38d0ccb2817c6d3f01e8b0c4792c77845feb43571092dcf5ebb5': 'interest_bearing_collateral',
    '0x00ba3037d968790ac486f70acaa9a1cab10cf5843bb85c986624b4d0e5a82e74': 'debt',
    '0x063d69ae657bd2f40337c39bf35a870ac27ddf91e6623c2f52529db4c1619a51': 'debt',
    '0x024e9b0d6bc79e111e6872bb1ada2a874c25712cf08dfc5bcf0de008a7cca55f': 'debt',
    '0x066037c083c33330a8460a65e4748ceec275bbf5f28aa71b686cbc0010e12597': 'debt',
    '0x0491480f21299223b9ce770f23a2c383437f9fbf57abc2ac952e9af8cdb12c97': 'debt',
    '0x0348cc417fc877a7868a66510e8e0d0f3f351f5e6b0886a86b652fcb30a3d1fb': 'debt',
    '0x035778d24792bbebcf7651146896df5f787641af9e2a3db06480a637fbc9fff8': 'debt',
    '0x001258eae3eae5002125bebf062d611a772e8aea3a1879b64a19f363ebd00947': 'debt',
    '0x0292be6baee291a148006db984f200dbdb34b12fb2136c70bfe88649c12d934b': 'debt',
}
NOSTRA_MAINNET_INTEREST_RATE_MODEL_ADDRESS: str = '0x059a943ca214c10234b9a3b61c558ac20c005127d183b86a99a8f3c60a08b4ff'

# Define the event type mapping based on the token type and key name
NOSTRA_MAINNET_TOKEN_TYPES_TO_EVENT_TYPES : dict[str, str] = {
    'non_interest_bearing_collateral': {
        'Mint': 'deposit',
        'Burn': 'withdrawal'
    },
    'interest_bearing_collateral': {
        'Mint': 'deposit',
        'Burn': 'withdrawal'
    },
    'debt': {
        'Mint': 'borrowing',
        'Burn': 'repayment'
    }
}
def get_nostra_mainnet_event_type(token_address: str, key_name: str) ->str:
    # Special case for InterestStateUpdated and the specific interest rate model address
    if token_address == NOSTRA_MAINNET_INTEREST_RATE_MODEL_ADDRESS and key_name == 'InterestStateUpdated':
        return 'interest_rate_updated'

    token_type = NOSTRA_MAINNET_TOKENS_TO_TOKEN_TYPES.get(token_address)
    if token_type:
        event_type = NOSTRA_MAINNET_TOKEN_TYPES_TO_EVENT_TYPES.get(token_type, {}).get(key_name)
        if event_type:
            return event_type
    # If no match is found, raise an error
    raise ValueError(f"Unrecognized event type for token address {token_address} and key name {key_name}")

def extract_nostra_mainnet(row: pd.Series) -> pd.Series:
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
        event_type = get_nostra_mainnet_event_type(row['from_address'], row['key_name'])
        
        if row['key_name'] in ['Mint', 'Burn']:
            return pd.Series({
                'user': data_list[0],
                'token': row['from_address'],
                'face_amount': hex_to_int(data_list[1]),
                'event_type': event_type
            })
        elif row['key_name'] == 'InterestStateUpdated':
            return pd.Series({
                'debt_token': data_list[0],
                'lending_rate': data_list[1],
                'borrow_rate': data_list[3],
                'lend_index': data_list[5],
                'borrow_index': data_list[7],
                'event_type': 'interest_rate_updated'
                
            })
        else:
            raise ValueError(f"Unrecognized key_name: {row['key_name']}") # Raising an error for unrecognized key_name
    except Exception as e:
        # Handle any parsing errors
        raise ValueError(f"Failed to extract data from row: {str(e)}") from e
# Filter the dataset for 'nostra_mainnet' protocol
nostra_mainnet_events = events[events['Protocol'] == 'nostra_mainnet']

# Apply the extraction function to each row
standardized_nostra_mainnet_events = nostra_mainnet_events.apply(extract_nostra_mainnet, axis=1)

# Concatenate the extracted elements with the original dataframe
standardized_nostra_mainnet_events = pd.concat([nostra_mainnet_events[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], standardized_nostra_mainnet_events], axis=1)

# Display the resulting dataframe
standardized_nostra_mainnet_events.head()

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'extracted_standard_mainnet_events.csv'
# standard_mainnet.to_csv(output_file_path, index=False)

,block_hash,block_number,transaction_hash,timestamp,key_name,borrow_index,borrow_rate,debt_token,event_type,face_amount,lend_index,lending_rate,token,user
200000,0x051eddd5b8502044947dc35d687caa2b2634d3dbb670d3f1f28654fffdcdc892,169098,0x00489af46e28392d1c3e4007476328ba4ccf4bd84f4f5565fda0888d5518a70b,1692858419,Mint,NaN,NaN,NaN,deposit,1000000000000000000,NaN,NaN,0x04f18ffc850cdfa223a530d7246d3c6fc12a5969e0aa5d4a88f470f5fe6c46e9,0x5295882956890bb9c05e54c1e7efa903a4a21083ebd6a3c175507f74e549574
200001,0x051eddd5b8502044947dc35d687caa2b2634d3dbb670d3f1f28654fffdcdc892,169098,0x00489af46e28392d1c3e4007476328ba4ccf4bd84f4f5565fda0888d5518a70b,1692858419,InterestStateUpdated,0xde0b6b3a7640000,0x0,0x66037c083c33330a8460a65e4748ceec275bbf5f28aa71b686cbc0010e12597,interest_rate_updated,NaN,0xde0b6b3a7640000,0x0,NaN,NaN
200002,0x0068928198ae9c350e91761752850081262aede8cabd5824341c905b5750fae5,169911,0x01bf3b09632626f8cc8ed46a1b394df8ddf2776b5f04794ad740e335b1ec5b9a,1692881265,Mint,NaN,NaN,NaN,deposit,100000,NaN,NaN,0x05dcd26c25d9d8fd9fc860038dcb6e4d835e524eb8a85213a8cda5b7fff845f6,0x2d50b82f53cc24f44d562207009f89f60b04ba98bff6deba999b50b964c9dfe
200003,0x0068928198ae9c350e91761752850081262aede8cabd5824341c905b5750fae5,169911,0x01bf3b09632626f8cc8ed46a1b394df8ddf2776b5f04794ad740e335b1ec5b9a,1692881265,InterestStateUpdated,0xde0b6b3a7640000,0x0,0x63d69ae657bd2f40337c39bf35a870ac27ddf91e6623c2f52529db4c1619a51,interest_rate_updated,NaN,0xde0b6b3a7640000,0x0,NaN,NaN
200004,0x02ce054477550a921fcd804daf2482d9503562dfb4790568aba6783184dd5656,175894,0x003e1ef7eca78dff0bf0e181658360ecf2a5f0d0380b5b9c1f239e3f2b057809,1693155334,Mint,NaN,NaN,NaN,deposit,25007406626554445,NaN,NaN,0x057146f6409deb4c9fa12866915dd952aa07c1eb2752e451d7f3b042086bdeb8,0x1b706b18846667c44fdf2dc302ed90b98c0ae3aa6e041fdbdea1289d0284b03
